In [46]:
import pandas as pd
import statsmodels.api as sm
import linearmodels as lm
import pyreadstat

In [47]:
data_stata, meta = pyreadstat.read_dta(r'/Users/dhruvpandit/Documents/GitHub/ISEG_AEF/Semester-Two-Projects/T1.dta')
label_dict = {}
for var_name, label in zip(meta.column_names, meta.column_labels):
    label_dict[var_name] = label


In [48]:
data_stata

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix,trend
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857,2.0
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732,3.0
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331,4.0
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,197,83,0.015575,0.226667,0.480392,0.428571,7.77,0.001073,0.869048,18.905853,...,1.697597,-0.148666,-0.010969,-0.127018,0.164303,0.157158,0.149330,0.070461,0.020250,3.0
626,197,84,0.013662,0.204188,1.410260,0.372727,10.11,0.001109,0.872024,22.704754,...,1.697597,-0.131037,-0.104441,1.076927,-0.139610,0.263255,0.032795,0.183103,0.026842,4.0
627,197,85,0.013086,0.180556,0.830769,0.333333,5.96,0.001054,0.875000,24.123611,...,1.697597,-0.043091,-0.123000,-0.529178,-0.111704,-0.528454,-0.050473,0.060617,-0.366374,5.0
628,197,86,0.012874,0.112676,2.250000,0.244444,7.68,0.001088,0.880952,24.981979,...,1.697597,-0.016311,-0.471524,0.996334,-0.310156,0.253549,0.031580,0.034964,-0.067911,6.0


In [49]:
data_stata['year'] = data_stata['year'].apply(lambda x: '19'+str(x))
data_stata

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix,trend
0,1,1981,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1982,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857,2.0
2,1,1983,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732,3.0
3,1,1984,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331,4.0
4,1,1985,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,197,1983,0.015575,0.226667,0.480392,0.428571,7.77,0.001073,0.869048,18.905853,...,1.697597,-0.148666,-0.010969,-0.127018,0.164303,0.157158,0.149330,0.070461,0.020250,3.0
626,197,1984,0.013662,0.204188,1.410260,0.372727,10.11,0.001109,0.872024,22.704754,...,1.697597,-0.131037,-0.104441,1.076927,-0.139610,0.263255,0.032795,0.183103,0.026842,4.0
627,197,1985,0.013086,0.180556,0.830769,0.333333,5.96,0.001054,0.875000,24.123611,...,1.697597,-0.043091,-0.123000,-0.529178,-0.111704,-0.528454,-0.050473,0.060617,-0.366374,5.0
628,197,1986,0.012874,0.112676,2.250000,0.244444,7.68,0.001088,0.880952,24.981979,...,1.697597,-0.016311,-0.471524,0.996334,-0.310156,0.253549,0.031580,0.034964,-0.067911,6.0


In [50]:
data_stata['year'] = pd.to_datetime(data_stata['year'], format='%Y')
data_stata

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix,trend
0,1,1981-01-01,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1982-01-01,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857,2.0
2,1,1983-01-01,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732,3.0
3,1,1984-01-01,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331,4.0
4,1,1985-01-01,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,197,1983-01-01,0.015575,0.226667,0.480392,0.428571,7.77,0.001073,0.869048,18.905853,...,1.697597,-0.148666,-0.010969,-0.127018,0.164303,0.157158,0.149330,0.070461,0.020250,3.0
626,197,1984-01-01,0.013662,0.204188,1.410260,0.372727,10.11,0.001109,0.872024,22.704754,...,1.697597,-0.131037,-0.104441,1.076927,-0.139610,0.263255,0.032795,0.183103,0.026842,4.0
627,197,1985-01-01,0.013086,0.180556,0.830769,0.333333,5.96,0.001054,0.875000,24.123611,...,1.697597,-0.043091,-0.123000,-0.529178,-0.111704,-0.528454,-0.050473,0.060617,-0.366374,5.0
628,197,1986-01-01,0.012874,0.112676,2.250000,0.244444,7.68,0.001088,0.880952,24.981979,...,1.697597,-0.016311,-0.471524,0.996334,-0.310156,0.253549,0.031580,0.034964,-0.067911,6.0


In [51]:
data_stata = data_stata[['county', 'year','crmrte', 'prbconv', 'prbarr', 'avgsen', 'polpc', 'density', 'taxpc', 'west', 'central', 'urban' ]]

In [52]:
data_stata.describe()

,county,crmrte,prbconv,prbarr,avgsen,polpc,density,taxpc,west,central,urban
count,630.00000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000
mean,100.60000,0.031588,0.688618,0.307368,8.954540,0.001917,1.386062,30.239194,0.233333,0.377778,0.088889
std,58.03627,0.018121,1.690345,0.171205,2.658082,0.002735,1.439703,11.454694,0.423289,0.485217,0.284809
min,1.00000,0.001812,0.068376,0.058823,4.220000,0.000459,0.197719,14.302565,0.000000,0.000000,0.000000
25%,51.00000,0.018352,0.347692,0.217902,7.160000,0.001191,0.532944,23.425596,0.000000,0.000000,0.000000
50%,103.00000,0.028441,0.474375,0.278240,8.495000,0.001451,0.952595,27.792328,0.000000,0.000000,0.000000
75%,151.00000,0.038406,0.635597,0.352518,10.197500,0.001803,1.507818,33.271218,0.000000,1.000000,0.000000
max,197.00000,0.163835,37.000000,2.750000,25.830000,0.035578,8.827652,119.761452,1.000000,1.000000,1.000000


In [53]:
data_df_multi = data_stata.set_index(['county', 'year'])

In [54]:
data_df_multi

crmrte   prbconv    prbarr  avgsen     polpc   density  \
county year                                                                   
1      1981-01-01  0.039885  0.402062  0.289696    5.61  0.001787  2.307159   
       1982-01-01  0.038345  0.433005  0.338111    5.59  0.001767  2.330254   
       1983-01-01  0.030305  0.525703  0.330449    5.80  0.001836  2.341801   
       1984-01-01  0.034726  0.604706  0.362525    6.89  0.001886  2.346420   
       1985-01-01  0.036573  0.578723  0.325395    6.55  0.001924  2.364896   
...                     ...       ...       ...     ...       ...       ...   
197    1983-01-01  0.015575  0.480392  0.226667    7.77  0.001073  0.869048   
       1984-01-01  0.013662  1.410260  0.204188   10.11  0.001109  0.872024   
       1985-01-01  0.013086  0.830769  0.180556    5.96  0.001054  0.875000   
       1986-01-01  0.012874  2.250000  0.112676    7.68  0.001088  0.880952   
       1987-01-01  0.014193  1.182930  0.207595   12.23  0.001186  0.889881   

                       taxpc  west  central  urban  
county year                                         
1      1981-01-01  25.697630     0        1      0  
       1982-01-01  24.874252     0        1      0  
       1983-01-01  26.451443     0        1      0  
       1984-01-01  26.842348     0        1      0  
       1985-01-01  28.140337     0        1      0  
...                      ...   ...      ...    ...  
197    1983-01-01  18.905853     1        0      0  
       1984-01-01  22.704754     1        0      0  
       1985-01-01  24.123611     1        0      0  
       1986-01-01  24.981979     1        0      0  
       1987-01-01  25.952581     1        0      0  

[630 rows x 10 columns]

In [55]:
obs_counts = data_stata.groupby('county')['year'].count()
if obs_counts.std() > 0: print('Unbalanced Data') 
else: print('Balanced')

Balanced


In [56]:
#creating dummy variables for the time periods
data_stata['year'] = data_stata['year'].dt.year
dummies = pd.get_dummies(data_stata['year'], )
data_stata = pd.concat([data_stata, dummies], axis = 1)
data_stata

/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_33160/558770162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_stata['year'] = data_stata['year'].dt.year


,county,year,crmrte,prbconv,prbarr,avgsen,polpc,density,taxpc,west,central,urban,1981,1982,1983,1984,1985,1986,1987
0,1,1981,0.039885,0.402062,0.289696,5.61,0.001787,2.307159,25.697630,0,1,0,1,0,0,0,0,0,0
1,1,1982,0.038345,0.433005,0.338111,5.59,0.001767,2.330254,24.874252,0,1,0,0,1,0,0,0,0,0
2,1,1983,0.030305,0.525703,0.330449,5.80,0.001836,2.341801,26.451443,0,1,0,0,0,1,0,0,0,0
3,1,1984,0.034726,0.604706,0.362525,6.89,0.001886,2.346420,26.842348,0,1,0,0,0,0,1,0,0,0
4,1,1985,0.036573,0.578723,0.325395,6.55,0.001924,2.364896,28.140337,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,197,1983,0.015575,0.480392,0.226667,7.77,0.001073,0.869048,18.905853,1,0,0,0,0,1,0,0,0,0
626,197,1984,0.013662,1.410260,0.204188,10.11,0.001109,0.872024,22.704754,1,0,0,0,0,0,1,0,0,0
627,197,1985,0.013086,0.830769,0.180556,5.96,0.001054,0.875000,24.123611,1,0,0,0,0,0,0,1,0,0
628,197,1986,0.012874,2.250000,0.112676,7.68,0.001088,0.880952,24.981979,1,0,0,0,0,0,0,0,1,0


In [64]:
exog = sm.add_constant(data_df_multi[['prbconv', 'prbarr', 'avgsen', 'polpc', 'density', 'taxpc', 'west', 'central', 'urban' ]] )

pooled_model  = lm.PooledOLS(data_df_multi['crmrte'], exog)
result = pooled_model.fit()

In [75]:
result

Dep. Variable:,crmrte,R-squared:,0.7037
Estimator:,PooledOLS,R-squared (Between):,0.7937
No. Observations:,630,R-squared (Within):,0.0977
Date:,"Fri, Mar 10 2023",R-squared (Overall):,0.7037
Time:,16:24:39,Log-likelihood,2016.4
Cov. Estimator:,Unadjusted,,
,,F-statistic:,163.57
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(9,620)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,163.57


In [65]:
print(result)

                          PooledOLS Estimation Summary                          
Dep. Variable:                 crmrte   R-squared:                        0.7037
Estimator:                  PooledOLS   R-squared (Between):              0.7937
No. Observations:                 630   R-squared (Within):               0.0977
Date:                Fri, Mar 10 2023   R-squared (Overall):              0.7037
Time:                        16:24:39   Log-likelihood                    2016.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      163.57
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                   F(9,620)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             163.57
                            

In [76]:
result_robust = pooled_model.fit(cov_type="robust")
result_robust

Dep. Variable:,crmrte,R-squared:,0.7037
Estimator:,PooledOLS,R-squared (Between):,0.7937
No. Observations:,630,R-squared (Within):,0.0977
Date:,"Fri, Mar 10 2023",R-squared (Overall):,0.7037
Time:,16:33:21,Log-likelihood,2016.4
Cov. Estimator:,Robust,,
,,F-statistic:,163.57
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(9,620)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,211.94


In [77]:
pooled_model.fit(
    cov_type="clustered", cluster_entity=True
)

Dep. Variable:,crmrte,R-squared:,0.7037
Estimator:,PooledOLS,R-squared (Between):,0.7937
No. Observations:,630,R-squared (Within):,0.0977
Date:,"Fri, Mar 10 2023",R-squared (Overall):,0.7037
Time:,16:33:31,Log-likelihood,2016.4
Cov. Estimator:,Clustered,,
,,F-statistic:,163.57
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(9,620)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,67.885


In [78]:
pooled_model.fit(
    cov_type="clustered", cluster_entity=True, cluster_time=True
)

Dep. Variable:,crmrte,R-squared:,0.7037
Estimator:,PooledOLS,R-squared (Between):,0.7937
No. Observations:,630,R-squared (Within):,0.0977
Date:,"Fri, Mar 10 2023",R-squared (Overall):,0.7037
Time:,16:33:35,Log-likelihood,2016.4
Cov. Estimator:,Clustered,,
,,F-statistic:,163.57
Entities:,90,P-value,0.0000
Avg Obs:,7.0000,Distribution:,"F(9,620)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,88.032


In [ ]:
result

In [68]:
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
# conduct Breusch-Pagan test and get results
bp_test = het_breuschpagan(result.resids, exog)
print("Breusch-Pagan test:\nLM-statistic: {:.3f}\np-value: {:.3f}\nf-value: {:.3f}\nf p-value: {:.3f}"
      .format(bp_test[0], bp_test[1], bp_test[2], bp_test[3]))


Breusch-Pagan test:
LM-statistic: 220.802
p-value: 0.000
f-value: 37.172
f p-value: 0.000


In [70]:
# conduct White test and get results
white_test = het_white(result.resids, exog)
print("White test\nLM-statistic: {:.3f}\np-value: {:.3f}\nf-value: {:.3f}\nf p-value: {:.3f}"
      .format(white_test[0], white_test[1], white_test[2], white_test[3]))


White test
LM-statistic: 359.772
p-value: 0.000
f-value: 15.417
f p-value: 0.000


In [87]:
re_model = lm.RandomEffects(data_df_multi['crmrte'], exog)
re_results = re_model.fit()

In [82]:
fe_model = lm.PanelOLS(data_df_multi['crmrte'], exog, entity_effects=True, drop_absorbed=True)
fe_results = fe_model.fit()

/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_33160/3034651821.py:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

west, central, urban

  fe_results = fe_model.fit()


In [85]:
fe_te_model = lm.PanelOLS(data_df_multi['crmrte'], exog, entity_effects=True,time_effects=True, drop_absorbed=True)
fe_te_results = fe_te_model.fit()

/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_33160/2268137198.py:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

west, central, urban

  fe_te_results = fe_te_model.fit()


In [88]:
from linearmodels.panel import compare

print(compare({"FE": fe_results, "RE": re_results, "Pooled": result, "FETE": fe_te_results}))

                                   Model Comparison                                   
                                    FE                RE         Pooled           FETE
--------------------------------------------------------------------------------------
Dep. Variable                   crmrte            crmrte         crmrte         crmrte
Estimator                     PanelOLS     RandomEffects      PooledOLS       PanelOLS
No. Observations                   630               630            630            630
Cov. Est.                   Unadjusted        Unadjusted     Unadjusted     Unadjusted
R-squared                       0.2435            0.4280         0.7037         0.2698
R-Squared (Within)              0.2435            0.2334         0.0977         0.2381
R-Squared (Between)             0.1575            0.7420         0.7937         0.3588
R-Squared (Overall)             0.1686            0.6762         0.7037         0.3432
F-statistic                     28.649     

In [93]:
# 3.B Non-Autocorrelation
# Durbin-Watson-Test
from statsmodels.stats.stattools import durbin_watson

durbin_watson_test_results = durbin_watson(result.resids) 
print(durbin_watson_test_results)

0.8816558678548779


In [95]:
# 3.B Non-Autocorrelation
# Durbin-Watson-Test
from statsmodels.stats.stattools import durbin_watson

durbin_watson_test_results = durbin_watson(re_results.resids) 
print(durbin_watson_test_results)

1.5615781006334277


In [ ]:
import numpy.linalg as la
from scipy import stats
import numpy as np
def hausman(fe, re):
 b = fe.params
 B = re.params
 v_b = fe.cov
 v_B = re.cov
df = b[np.abs(b) < 1e8].size
chi2 = np.dot((b — B).T, la.inv(v_b — v_B).dot(b — B)) 
 
 pval = stats.chi2.sf(chi2, df)
return chi2, df, pval
hausman_results = hausman(fe_res, re_res) 
print(‘chi-Squared: ‘ + str(hausman_results[0]))
print(‘degrees of freedom: ‘ + str(hausman_results[1]))
print(‘p-Value: ‘ + str(hausman_results[2]))